## <span style="color:purple">Morphological analysis (stem-based)</span>

By default, Vabamorf's morphological analysis is _lemma-based_, which means that inflectional forms will be normalized to their lemmas in the output. For instance, verb _'läks'_ will be normalized to root/lemma _'minema'_ and noun _'vette'_ will be normalized to root/lemma _'vesi'_. The default _lemma-based_ morphological analysis is described in tutorial [01_morphological_analysis.ipynb](01_morphological_analysis.ipynb). 
However, Vabamorf can also produce a **stem-based morphological analysis**, in which inflectional forms will be kept as they are in the surface form. For instance, verb _'läks'_ will be normalized to root/stem _'läk'_ (with ending _'s'_) and noun _'vette'_ will be normalized to root/stem _'ve'_ (with ending _'tt'_). 

Stem-based analysis can be switched on in VabamorfTagger (and in its sub-components VabamorfAnalyzer, PostMorphAnalysisTagger and VabamorfDisambiguator) by passing the flag `stem=True` to the tagger:

In [1]:
from estnltk import Text
from estnltk.taggers import VabamorfTagger

# Initialize stem-based morphological analyser
# Also, rename the layer to distinguish it from the default morph layer
morph_tagger = VabamorfTagger(output_layer='stembased_morph_analysis', stem=True)

# Prepare text
text=Text("Ta läks omastega rappa ja jõi hundijalavett.")
# Tag layers required by morph analysis
text.tag_layer(['words','sentences'])

# Tag morph analysis
morph_tagger.tag( text )

# Results
text.stembased_morph_analysis

Layer(name='stembased_morph_analysis', attributes=('normalized_text', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Ta', [{'normalized_text': 'Ta', 'root': 'ta', 'root_tokens': ['ta'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P'}]),
Span('läks', [{'normalized_text': 'läks', 'root': 'läk', 'root_tokens': ['läk'], 'ending': 's', 'clitic': '', 'form': 's', 'partofspeech': 'V'}]),
Span('omastega', [{'normalized_text': 'omastega', 'root': 'omas', 'root_tokens': ['omas'], 'ending': 'tega', 'clitic': '', 'form': 'pl kom', 'partofspeech': 'S'}]),
Span('rappa', [{'normalized_text': 'rappa', 'root': 'rappa', 'root_tokens': ['rappa'], 'ending': '0', 'clitic': '', 'form': 'adt', 'partofspeech': 'S'}]),
Span('ja', [{'normalized_text': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('jõi', [{'normalized_text': 'jõi', 'root': 'jõ', 'root_tokens': ['jõ'], 'ending': 'i', 'clitic': '', 'form': 's', 'partofspeech': 'V'}]),
Span('hundijalavett', [{'normalized_text': 'hundijalavett', 'root': 'hundi_jala_ve', 'root_tokens': ['hundi', 'jala', 've'], 'ending': 'tt', 'clitic': '', 'form': 'sg p', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Note that there is no _lemma_ attribute in the output layer. 
This is also hinders the usability of the `stembased_morph_analysis` by other tools, because most of the tools require lemma-based morphological analysis.

Let's now compare stem-based and lemma-based morphological analysis:

In [2]:
# add the lemma-based morphological analysis
text.tag_layer('morph_analysis')

# compare roots of stem-based and lemma-based morphological analysis
for word, lemma, stem, ending in zip(text.words.text, text.morph_analysis['root'], \
                                     text.stembased_morph_analysis['root'], text.stembased_morph_analysis['ending']):
    print(f'{word!r:15}  lemma-based root: {lemma!r:20}  stem-based root: {stem!r:20}  ending: {ending!r}')

'Ta'             lemma-based root: ['tema']              stem-based root: ['ta']                ending: ['0']
'läks'           lemma-based root: ['mine']              stem-based root: ['läk']               ending: ['s']
'omastega'       lemma-based root: ['omaksed']           stem-based root: ['omas']              ending: ['tega']
'rappa'          lemma-based root: ['raba']              stem-based root: ['rappa']             ending: ['0']
'ja'             lemma-based root: ['ja']                stem-based root: ['ja']                ending: ['0']
'jõi'            lemma-based root: ['joo']               stem-based root: ['jõ']                ending: ['i']
'hundijalavett'  lemma-based root: ['hundi_jala_vesi']   stem-based root: ['hundi_jala_ve']     ending: ['tt']
'.'              lemma-based root: ['.']                 stem-based root: ['.']                 ending: ['']


### Limitations

<div class="alert alert-block alert-warning">
<h4><i>VabamorfTagger's limitations with stem-based analysis</i></h4>
<br>
VabamorfTagger's flags <code>predisambiguate</code>, <code>postdisambiguate</code> and <code>use_reorderer</code> will be automatically set to <code>False</code> with stem-based analysis, because corresponding components (<code>CorpusBasedMorphDisambiguator</code> and <code>MorphAnalysisReorderer</code>) <b>do not support</b> stem-based analysis.
</div>

<div class="alert alert-block alert-warning">
<h4><i>Warning: Stem-based morphological analysis <b>is not supported</b> by most of the downstream taggers in the pipeline</i></h4>
<br>
Please note that stem-based morphological analysis is currently <b>not supported</b> by most of downstream taggers that are depending on morphological analysis (such as named entity recognition and syntactic parsing). 
The taggers become unapplicable if the <code>morph_analysis</code> layer contains stem-based analysis instead of the default lemma-based one. Therefore, if you use stem-based morphological analysis, it is not recommended to use any taggers depending on <code>morph_analysis</code> (or use these taggers only when you know what you are doing).
</div>